# remove the combined mask at first 


In [18]:
import os
import glob 
combined_scan = glob.glob('New/s*/*/combined*')
print(len(combined_scan ))
# remove these files
for file in combined_scan:
    os.remove(file)
combined_scan = glob.glob('New/s*/*/combined*')
print(len(combined_scan ))

11
0


In [15]:
combined_scan = glob.glob('New/s0001/segmentations/*')
len(combined_scan)

117

In [16]:
import pandas as pd

# Load the Excel file
df = pd.read_excel('label_names.xlsx')

# Extract the multiplied_labels column
multiplied_labels = df['multiplied_labels'].tolist()

# Check for duplicates
duplicates = [label for label in multiplied_labels if multiplied_labels.count(label) > 1]
if duplicates:
    print("Duplicate labels found:", set(duplicates))
else:
    print("No duplicate labels found.")

# Check the maximum label value
max_label = max(multiplied_labels)
print("Maximum label value:", max_label)

# Check the number of unique labels
unique_labels = set(multiplied_labels)
print("Number of unique labels:", len(unique_labels))


No duplicate labels found.
Maximum label value: 117
Number of unique labels: 117


In [2]:
import pandas as pd
import glob
import nibabel as nib
import numpy as np
import tqdm
import os 

# Load label information from Excel
df = pd.read_excel('label_names.xlsx')
label_names = df['label_names'].tolist()
multiplied_labels = df['multiplied_labels'].tolist()
print(label_names[0], len(label_names))
print(multiplied_labels[0], len(multiplied_labels))

# Get list of folders containing masks
folders = glob.glob('New/s*/se*')
folders.sort()

for folder_num, folder in enumerate(tqdm.tqdm(folders, desc="Processing folders")):
    # Load one mask to get the shape and affine information
    initial_file = os.path.join(folder, label_names[0])
    initial_mask_nii = nib.load(initial_file)
    mask_shape = initial_mask_nii.get_fdata().shape
    affine = initial_mask_nii.affine

    # Initialize the combined mask with zeros
    combined_mask = np.zeros(mask_shape)
    non_empty_label_count = 0
    num_masks = []

    for label_num, (label_name, label_value) in enumerate(zip(label_names, multiplied_labels)):
        file_path = os.path.join(folder, label_name)
        comb_file_path = os.path.join(folder, f'combined_mask.nii.gz')

        file = nib.load(file_path)
        mask = file.get_fdata()
        combined_scan = nib.load(comb_file_path).get_fdata()

        
        
        if mask.sum() == 0:
            continue
        else:
            non_empty_label_count += 1
            num_masks.append(label_value)
    
    #print(num_masks)
    #print(np.unique(combined_scan))
    # get the unique values in between combined mask and num_masks
    unique_values = np.unique(combined_scan)
    unique_values = np.setdiff1d(unique_values, num_masks)
    #print()
    
    if len(unique_values) > 1:
        print(folder, len(np.unique(combined_scan)), unique_values)
    #print(non_empty_label_count, len(np.unique(combined_scan)), unique_values)

        


adrenal_gland_left.nii.gz 117
1 117


Processing folders:   0%|          | 0/1228 [00:00<?, ?it/s]

Processing folders: 100%|██████████| 1228/1228 [7:11:35<00:00, 21.09s/it]  


In [15]:
len(glob.glob('dataset_nii/val/labels/*'))

89

In [12]:
len(file)

1171

In [8]:
import os 
import nibabel as nib
import numpy as np
import glob
from tqdm import tqdm

file = glob.glob('dataset_nii/*/labels/*')
for f in tqdm(file):
    mask = nib.load(f).get_fdata()
    number = np.unique(mask)
    if 117 in number:
        print(f, number)

100%|██████████| 1171/1171 [08:58<00:00,  2.17it/s]


In [24]:
import pandas as pd
df = pd.read_excel('meta.xlsx')
df1 = df[df['split']=='train']
print(len(df1))
df2 = df[df['split']=='val']
print(len(df2))
len(df1) + len(df2)
df['split'].value_counts()

1082
57


split
train    1082
test       89
val        57
Name: count, dtype: int64

In [25]:
# read the mketa.xlsx file 
import pandas as pd
df = pd.read_excel('meta.xlsx')
# select only train values in split column
df = df[df['split']=='train']
print(df.columns)
# make directories for train and test
import os
os.makedirs('dataset_nii/train/images', exist_ok=True)
os.makedirs('dataset_nii/train/labels', exist_ok=True)


# get the name of images and labels
files = df['image_id'].values
# from these folders get the images and labels
image_folder = 'New'
label_folder = 'segmentations' 
import glob 
import tqdm
for i in tqdm.tqdm(range(len(files))):

    images = 'New/'+files[i]+'/ct.nii.gz'
    labels = 'New/'+files[i]+'/segmentations/combined_mask.nii.gz'
    # copy the images, change the name of the images sand then save them in the train folder
    import shutil
    shutil.copy(images, 'dataset_nii/train/images/'+files[i]+'.nii.gz') 
    shutil.copy(labels, 'dataset_nii/train/labels/'+files[i]+'.nii.gz')




Index(['image_id', 'age', 'gender', 'institute', 'study_type', 'split',
       'manufacturer', 'scanner_model', 'kvp', 'pathology',
       'pathology_location'],
      dtype='object')


100%|██████████| 1082/1082 [00:13<00:00, 82.04it/s] 


In [26]:
# read the mketa.xlsx file 
import pandas as pd
df = pd.read_excel('meta.xlsx')
# select only train values in split column
df = df[df['split']=='test']
print(df.columns)
# make directories for train and test
import os

os.makedirs('dataset_nii/test/images', exist_ok=True)
os.makedirs('dataset_nii/test/labels', exist_ok=True)

# get the name of images and labels
files = df['image_id'].values
# from these folders get the images and labels
image_folder = 'New'
label_folder = 'segmentations' 
import glob 
import tqdm
for i in tqdm.tqdm(range(len(files))):

    images = 'New/'+files[i]+'/ct.nii.gz'
    labels = 'New/'+files[i]+'/segmentations/combined_mask.nii.gz'
    # copy the images, change the name of the images sand then save them in the train folder
    import shutil
    shutil.copy(images, 'dataset_nii/test/images/'+files[i]+'.nii.gz') 
    shutil.copy(labels, 'dataset_nii/test/labels/'+files[i]+'.nii.gz')




Index(['image_id', 'age', 'gender', 'institute', 'study_type', 'split',
       'manufacturer', 'scanner_model', 'kvp', 'pathology',
       'pathology_location'],
      dtype='object')


100%|██████████| 89/89 [00:01<00:00, 63.75it/s]


In [27]:
# read the mketa.xlsx file 
import pandas as pd
df = pd.read_excel('meta.xlsx')
# select only train values in split column
df = df[df['split']=='val']
print(df.columns)
# make directories for train and test
import os

os.makedirs('dataset_nii/val/images', exist_ok=True)
os.makedirs('dataset_nii/val/labels', exist_ok=True)

# get the name of images and labels
files = df['image_id'].values
# from these folders get the images and labels
image_folder = 'New'
label_folder = 'segmentations' 
import glob 
import tqdm
for i in tqdm.tqdm(range(len(files))):

    images = 'New/'+files[i]+'/ct.nii.gz'
    labels = 'New/'+files[i]+'/segmentations/combined_mask.nii.gz'
    # copy the images, change the name of the images sand then save them in the train folder
    import shutil
    shutil.copy(images, 'dataset_nii/val/images/'+files[i]+'.nii.gz') 
    shutil.copy(labels, 'dataset_nii/val/labels/'+files[i]+'.nii.gz')




Index(['image_id', 'age', 'gender', 'institute', 'study_type', 'split',
       'manufacturer', 'scanner_model', 'kvp', 'pathology',
       'pathology_location'],
      dtype='object')


100%|██████████| 57/57 [00:00<00:00, 79.91it/s]


In [13]:
print(len('datase_nii/train/images/') == len('datase_nii/train/labels/'), len('datase_nii/test/images/') == len('datase_nii/test/labels/'))

True True


In [4]:
# preprocess CT scan 
import nibabel as nib
import os 
import numpy as np
import glob 
import tqdm
def preprocess_ct_scan(ct_scan_path):
    # Load the CT scan
    ct_scan = nib.load(ct_scan_path)
    affine = ct_scan.affine
    # Get the pixel values
    ct_scan_data = ct_scan.get_fdata()
    
    # Clip the pixel values to the range [0, 80]
    ct_scan_data = np.clip(ct_scan_data, 0, 80)
    
    # save it as nifti file
    ct_scan = nib.Nifti1Image(ct_scan_data, affine)
    # save the file
    nib.save(ct_scan, ct_scan_path)

ct_files = glob.glob('dataset_nii/train/images/*')

for ct_file in tqdm.tqdm(ct_files):
    preprocess_ct_scan(ct_file)



100%|██████████| 1082/1082 [12:41<00:00,  1.42it/s]


In [6]:
import m_yolo
ct_files = glob.glob('dataset_nii/val/images/*')

for ct_file in tqdm.tqdm(ct_files):
    m_yolo.preprocess.preprocess_ct_scan(ct_file)

import m_yolo
ct_files = glob.glob('dataset_nii/test/images/*')

for ct_file in tqdm.tqdm(ct_files):
    m_yolo.preprocess.preprocess_ct_scan(ct_file)



  0%|          | 0/57 [00:00<?, ?it/s]


AttributeError: module 'm_yolo' has no attribute 'preprocess'